In [1]:
from smart_open import open
import pandas as pd
import datetime
import numpy as np

In [457]:
data = pd.read_csv("../data/training_pruned_file_20162021.csv")

In [458]:
data = data.drop([
    "continent", 
    "player_id_transfermt", 
    "date", 
    "position_id_y", 
    "country_name", 
    "gender", 
    "common_name", 
    "firstname_x", 
    "lastname_x", 
    "name", 
    "height_x", 
    "weight_x", 
    "league.season", 
    "agent_name", 
    "global_performance",
    "birth.country",
    "nationality",
], axis=1)

In [459]:
set(data["position"].values)

{'Attack', 'Defender', 'Goalkeeper', 'Midfield'}

In [479]:
attackers = set(data[lambda x: x["position"] == "Attack"]["player_id_api"].unique())
defenders = set(data[lambda x: x["position"] == "Defender"]["player_id_api"].unique())
goalkeepers = set(data[lambda x: x["position"] == "Goalkeeper"]["player_id_api"].unique())
midfielders = set(data[lambda x: x["position"] == "Midfield"]["player_id_api"].unique())
len(data), len(attackers), len(defenders), len(goalkeepers), len(midfielders)

(14773, 1863, 1886, 669, 1306)

In [462]:
data["age"] = pd.to_datetime(data["date_of_birth"]).map(lambda x: int((pd.Timestamp.today() - x).days / 365))
del data["date_of_birth"]

In [463]:
data[lambda x: x["player_id_api"] == 154]

,player_id_api,team_name,season_name,league_name,accurate_crosses.total,accurate_passes.total,aerials_won.total,bench.total,big_chances_created.total,big_chances_missed.total,...,penalty.missed,penalty.saved,position,sub_position,foot,market_value_in_eur_x,highest_market_value_in_eur,position_name_x,position_name_y,age
2083,154,FC Barcelona,2017,La Liga,NaN,NaN,NaN,4.0,NaN,NaN,...,2.0,NaN,Attack,Right Winger,Left,180000000,180000000.0,Attacker,Right Wing,35
5254,154,FC Barcelona,2018,La Liga,12.0,1168.0,NaN,5.0,NaN,NaN,...,1.0,NaN,Attack,Right Winger,Left,150000000,180000000.0,Attacker,Right Wing,35
7466,154,FC Barcelona,2019,La Liga,27.0,1349.0,NaN,1.0,NaN,NaN,...,0.0,NaN,Attack,Right Winger,Left,112000000,180000000.0,Attacker,Right Wing,35
10914,154,FC Barcelona,2020,La Liga,35.0,1914.0,8.0,2.0,NaN,NaN,...,2.0,NaN,Attack,Right Winger,Left,80000000,180000000.0,Attacker,Right Wing,35
13566,154,Paris Saint Germain,2021,Ligue 1,8.0,1390.0,5.0,2.0,17.0,11.0,...,0.0,NaN,Attack,Right Winger,Left,50000000,180000000.0,Attacker,Right Wing,35


In [464]:
before_after_seasons = pd.merge(data, data, on="player_id_api", suffixes=["_before", "_after"]).query("season_name_after - season_name_before == 1")

for c in [
    "foot_after",
    "age_before",
    "season_name_after",
    "season_name_before",
]:
    del before_after_seasons[c]
    
for c in [
    "games.position",
    "highest_market_value_in_eur",
    "league.country",
    "league_name",
    "position",
    "position_name_x",
    "position_name_y",
    "sub_position",
    "team_name",
]:
    before_after_seasons[f"same_{c}"] = before_after_seasons[f"{c}_before"] == before_after_seasons[f"{c}_after"]
    del before_after_seasons[f"{c}_before"]
    del before_after_seasons[f"{c}_after"]


In [465]:
before_after_seasons = before_after_seasons[[d[0] for d in before_after_seasons.dtypes.items() if d[1] != object]]

In [466]:
def label(mv_diff):
    if mv_diff > 0:
        return "inc"
    elif mv_diff < 0:
        return "dec"
    else:
        return "same"
before_after_seasons["label"] = (before_after_seasons["market_value_in_eur_x_after"] - before_after_seasons["market_value_in_eur_x_before"]).map(label)
del before_after_seasons["market_value_in_eur_x_after"]
del before_after_seasons["market_value_in_eur_x_before"]

In [467]:
columns = {
    c
    for c in before_after_seasons.columns
    if "same_" not in c
}
columns.remove("label")

In [468]:
for c in columns:
    before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
before_after_seasons = before_after_seasons.copy()

/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This 

In [564]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(before_after_seasons.query("player_id_api in @goalkeepers").iloc[0].sort_index())

accurate_crosses.total_after                 NaN
accurate_crosses.total_before                NaN
accurate_passes.total_after                 28.0
accurate_passes.total_before                23.0
aerials_won.total_after                      NaN
aerials_won.total_before                     NaN
age_after                                     35
bench.total_after                           26.0
bench.total_before                          30.0
big_chances_created.total_after              NaN
big_chances_created.total_before             NaN
big_chances_missed.total_after               NaN
big_chances_missed.total_before              NaN
blocked_shots.total_after                    NaN
blocked_shots.total_before                   NaN
captain.total_after                          NaN
captain.total_before                         NaN
cards.red_after                              0.0
cards.red_before                             0.0
cards.yellow_after                           0.0
cards.yellow_before 

In [471]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [482]:
player_ids_api = before_after_seasons["player_id_api"]

In [610]:
y = before_after_seasons["label"]
# X = before_after_seasons.drop(["label", "player_id_api"], axis=1).fillna(0)
X = before_after_seasons[[
    "player_id_api",
    "age_after",
    # "has_games.minutes_before",
    # "has_games.minutes_after",
    # "games.minutes_before",
    # "games.minutes_after",
    # "has_minutes_played.total_before",
    # "has_minutes_played.total_after",
    # "minutes_played.total_before",
    # "minutes_played.total_after",
    # "goals.goals_before",
    # "goals.goals_after",
    # "total_duels.total_before",
    # "total_duels.total_after",
    "has_bench.total_before",
    "has_bench.total_after",
    "has_goals.saves_before",
    "has_goals.saves_after",
    "has_goals.conceded_before",
    "has_goals.conceded_after",
    "has_saves_inside_box.total_before",
    "has_saves_inside_box.total_after",
    "bench.total_before",
    "bench.total_after",
    "goals.saves_before",
    "goals.saves_after",
    "goals.conceded_before",
    "goals.conceded_after",
    "saves_inside_box.total_before",
    "saves_inside_box.total_after",
    "same_league.country",
    "same_league_name",
    "same_team_name",
]].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [549]:
len(X)

8910

In [594]:
y_train.value_counts() / len(y_train)

label
dec     0.576319
inc     0.306678
same    0.117003
Name: count, dtype: float64

In [231]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.dummy import DummyClassifier

In [607]:
if "label" in X_test:
    del X_test["label"]
if "pred" in X_test:
    del X_test["pred"]

pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('lr', LogisticRegression(random_state=42))
    # ('svc', SVC())
    ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train.drop("player_id_api", axis=1), y_train)

y_pred = pipe.predict(X_test.drop("player_id_api", axis=1))
print(
    "random",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test.drop("player_id_api", axis=1), y_test),
    sep="\n"
)

X_test["label"] = y_test
X_test["pred"] = y_pred
attackers_test = X_test.query("player_id_api in @attackers")
defenders_test = X_test.query("player_id_api in @defenders")
midfielders_test = X_test.query("player_id_api in @midfielders")
goalkeepers_test = X_test.query("player_id_api in @goalkeepers")

print([balanced_accuracy_score(df["label"], df["pred"]) for df in (attackers_test, defenders_test, midfielders_test, goalkeepers_test)])

random
confusion_matrix
[[181  75 327]
 [ 60  16 128]
 [337 114 544]]
balanced_accuracy_score
0.3118760542248682
accuracy_score
0.4158249158249158
pipe.score
0.42592592592592593
[0.3078738531176002, 0.31290933364440116, 0.30819490586932446, 0.3190803019903912]


In [611]:
if "label" in X_test:
    del X_test["label"]
if "pred" in X_test:
    del X_test["pred"]
    
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42))
    # ('svc', SVC())
    # ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train.drop("player_id_api", axis=1), y_train)

y_pred = pipe.predict(X_test.drop("player_id_api", axis=1))
print(
    "lr",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test.drop("player_id_api", axis=1), y_test),
    sep="\n"
)

X_test["label"] = y_test
X_test["pred"] = y_pred
attackers_test = X_test.query("player_id_api in @attackers")
defenders_test = X_test.query("player_id_api in @defenders")
midfielders_test = X_test.query("player_id_api in @midfielders")
goalkeepers_test = X_test.query("player_id_api in @goalkeepers")

print([balanced_accuracy_score(df["label"], df["pred"]) for df in (attackers_test, defenders_test, midfielders_test, goalkeepers_test)])

lr
confusion_matrix
[[276   1 306]
 [ 50   3 151]
 [141   3 851]]
balanced_accuracy_score
0.44779854777874845
accuracy_score
0.6341189674523008
pipe.score
0.6341189674523008
[0.42365094048720325, 0.44208852372117674, 0.4624223431744461, 0.5029989323572027]


In [612]:
if "label" in X_test:
    del X_test["label"]
if "pred" in X_test:
    del X_test["pred"]
    
selected_train = X_train.query("player_id_api in @goalkeepers").index
selected_test = X_test.query("player_id_api in @goalkeepers").index
    
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42))
    # ('svc', SVC())
    # ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(
    X_train.loc[selected_train].drop("player_id_api", axis=1), 
    y_train.loc[selected_train]
)

y_pred = pipe.predict(X_test.drop("player_id_api", axis=1))
print(
    "lr",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test.drop("player_id_api", axis=1), y_test),
    sep="\n"
)

X_test["label"] = y_test
X_test["pred"] = y_pred
attackers_test = X_test.query("player_id_api in @attackers")
defenders_test = X_test.query("player_id_api in @defenders")
midfielders_test = X_test.query("player_id_api in @midfielders")
goalkeepers_test = X_test.query("player_id_api in @goalkeepers")

print([balanced_accuracy_score(df["label"], df["pred"]) for df in (attackers_test, defenders_test, midfielders_test, goalkeepers_test)])

lr
confusion_matrix
[[226 288  69]
 [ 39 114  51]
 [140 622 233]]
balanced_accuracy_score
0.3935481564821382
accuracy_score
0.32154882154882153
pipe.score
0.32154882154882153
[0.4383426712978316, 0.37099120310253736, 0.39854229008113223, 0.5056966369251887]


In [521]:
if "label" in X_test:
    del X_test["label"]
if "pred" in X_test:
    del X_test["pred"]
    
pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('lr', LogisticRegression(random_state=42))
    ('svc', SVC())
    # ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train.drop("player_id_api", axis=1), y_train)

y_pred = pipe.predict(X_test.drop("player_id_api", axis=1))
print(
    "svc",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test.drop("player_id_api", axis=1), y_test),
    sep="\n"
)

X_test["label"] = y_test
X_test["pred"] = y_pred
attackers_test = X_test.query("player_id_api in @attackers")
defenders_test = X_test.query("player_id_api in @defenders")
midfielders_test = X_test.query("player_id_api in @midfielders")
goalkeepers_test = X_test.query("player_id_api in @goalkeepers")

print([balanced_accuracy_score(df["label"], df["pred"]) for df in (attackers_test, defenders_test, midfielders_test, goalkeepers_test)])

svc
confusion_matrix
[[347   0 236]
 [ 54   2 148]
 [115   0 880]]
balanced_accuracy_score
0.4964744292320018
accuracy_score
0.6896745230078564
pipe.score
0.6896745230078564
[0.4806968855997533, 0.48861485811002464, 0.5252845126175161, 0.48795470144131775]
